In [ ]:
from awpy.parser import DemoParser
from awpy.visualization.plot import plot_round
from dataclasses import dataclass, field, fields
from dataclasses_json import dataclass_json
import csv

In [ ]:
demo_file_name = r"demos/cloud9-vs-outsiders-m1-dust2.dem"

p = DemoParser(demofile=demo_file_name, parse_rate=128, parse_frames=True)

data = p.parse()

In [ ]:
@dataclass_json
@dataclass
class Position:
    """
    x, y, and z coordinates in a CSGO map.
    """
    x: float
    y: float
    z: float

@dataclass_json
@dataclass
class PlayerInfo:
    """
    Contains information about a CSGO player including their name, their team, and a list of their positions throughout the gme
    """
    name: str
    team: str
    positions: list[Position] # Length of this list is the number of frames in the round

@dataclass_json
@dataclass
class RoundInfo:
    """
    Contains a list of PlayerInfo objects, one for each player in the CSGO game round.
    """
    players: list[PlayerInfo] = field(default_factory=list)# Length of this list is the number of rounds in the game

@dataclass_json
@dataclass
class Game:
    """
    Contains a list of RoundInfo objects, one for each round in the CSGO game.
    """
    round_info: list[RoundInfo] = field(default_factory=list) # Length of this list is the number of rounds in the game

    def add_player_info(self, round_index: int, player_name: str, team:str, position: Position):
        """
        Uses the given information to construct a PlayerInfo object and add it to the RoundInfo object indexed by round_index
        """
        while len(self.round_info) <= round_index:
            self.round_info.append(RoundInfo())
        for player in self.round_info[round_index].players:
            if player.name == player_name:
                player.positions.append(position)
                return
        self.round_info[round_index].players.append(PlayerInfo(player_name, team, [position]))
        return

game = Game()

@dataclass_json
@dataclass
class PlayerState:
    """
    """
    round_index: int
    player_name: str
    team: str
    x: float
    y: float
    z: float

flat_game_info: list[PlayerState] = []

for game_round_index in range(0, len(data["gameRounds"])):
    frames = data["gameRounds"][game_round_index]["frames"]
    # print("Round: ", game_round_index + 1)
    for index, frame in enumerate(frames):
        # print("Frame: ", index + 1)
        for team in ["ct", "t"]:
            for player in frame[team]["players"]:
                position = Position(player["x"], player["y"], player["z"])
                # print(player["name"], position)
                game.add_player_info(game_round_index, player["name"], team, position)
                flat_game_info.append(PlayerState(game_round_index, player["name"], team, player["x"], player["y"], player["z"]))

def debug_print_game(game: Game):
    print("Round count: ", len(game.round_info))
    for round in game.round_info:
        print("Player count: ", len(round.players))
        for player in round.players:
            print("Frame count: ", len(player.positions))

# print(game.to_json())
print(flat_game_info)



In [ ]:
with open("data.csv", "w") as output_file:
    writer = csv.writer(output_file)
    writer.writerow([class_field.name for class_field in fields(PlayerState)])
    for state_entry in flat_game_info:
        writer.writerow([getattr(state_entry, field_name) for field_name in [class_field.name for class_field in fields(PlayerState)]])

In [ ]:
d = data
plot_round("best_round_ever.gif", d["gameRounds"][7]["frames"], map_name=d["mapName"], map_type="simpleradar", dark=False)

In [1]:
import parsing

demo_file_name = r"D:/Samihan/Documents/Programming/Python Projects/CSGO Demo Parsing/demos/cloud9-vs-outsiders-m1-dust2.dem"
print(f"Parsing {demo_file_name}")
data = parsing.parse_demo_file(demo_file_name)
print(data.keys())

Parsing D:/Samihan/Documents/Programming/Python Projects/CSGO Demo Parsing/demos/cloud9-vs-outsiders-m1-dust2.dem
dict_keys(['matchID', 'clientName', 'mapName', 'tickRate', 'playbackTicks', 'playbackFramesCount', 'parsedToFrameIdx', 'parserParameters', 'serverVars', 'matchPhases', 'matchmakingRanks', 'playerConnections', 'gameRounds'])


In [3]:
rounds = data["gameRounds"]
first_round = rounds[0]
for key in first_round.keys():
    print(key, type(first_round[key]))

roundNum <class 'int'>
isWarmup <class 'bool'>
startTick <class 'int'>
freezeTimeEndTick <class 'int'>
endTick <class 'int'>
endOfficialTick <class 'int'>
bombPlantTick <class 'NoneType'>
tScore <class 'int'>
ctScore <class 'int'>
endTScore <class 'int'>
endCTScore <class 'int'>
ctTeam <class 'str'>
tTeam <class 'str'>
winningSide <class 'str'>
winningTeam <class 'str'>
losingTeam <class 'str'>
roundEndReason <class 'str'>
ctFreezeTimeEndEqVal <class 'int'>
ctRoundStartEqVal <class 'int'>
ctRoundSpendMoney <class 'int'>
ctBuyType <class 'str'>
tFreezeTimeEndEqVal <class 'int'>
tRoundStartEqVal <class 'int'>
tRoundSpendMoney <class 'int'>
tBuyType <class 'str'>
ctSide <class 'dict'>
tSide <class 'dict'>
kills <class 'list'>
damages <class 'list'>
grenades <class 'list'>
bombEvents <class 'list'>
weaponFires <class 'list'>
flashes <class 'list'>
frames <class 'list'>


In [ ]:
frames = first_round["frames"]
first_frame = frames[0]
for key in first_frame.keys():
    print(key, type(first_frame[key]))